<a href="https://colab.research.google.com/github/maneeha/KGLLM/blob/main/Diabetes%20-%20%20Text%20to%20graph%20-%20RAG_With_Knowledge_graph(Neo4j)%20in%20excel%20sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# langchain-core

contains simple, core abstractions that have emerged as a standard, as well as LangChain Expression Language as a way to compose these components together. This package is now at version 0.1 and all breaking changes will be accompanied by a minor version bump.

# langchain-community
contains all third party integrations. We will work with partners on splitting key integrations out into standalone packages over the next month.

# langchain
contains higher-level and use-case specific chains, agents, and retrieval algorithms that are at the core of your application's cognitive architecture. We are targeting a launch of a stable 0.1 release for langchain in early January.#

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.8/384.8 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 20.0 MB/s eta 0:00:00
   

In [2]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [4]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [5]:
from typing import Tuple, List, Optional

In [6]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [7]:
from langchain_core.runnables import ConfigurableField

In [8]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


In [9]:
import os

In [10]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [11]:
from langchain_community.vectorstores import Neo4jVector

In [12]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [13]:
NEO4J_URI="neo4j+s://2ff1ce5d.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="jaCuHWs8uM_CmCjPhp99hukq8pwbt-ud5B9pZ-qIjtQ"

In [14]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [15]:
from langchain_community.graphs import Neo4jGraph

In [16]:
graph = Neo4jGraph()

In [17]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [18]:
from langchain.document_loaders import TextLoader

In [22]:
file_paths = ["/content/drive/MyDrive/refined/20121CoffmanNortonBeene.txt", "/content/drive/MyDrive/refined/insulin.txt", "/content/drive/MyDrive/refined/Blood gulocose monitoring.txt", "/content/drive/MyDrive/refined/blood gulocose.txt", "/content/drive/MyDrive/refined/complications.txt", "/content/drive/MyDrive/refined/diabetes awareness_TXT.txt", "/content/drive/MyDrive/refined/diabetes exercise.txt", "/content/drive/MyDrive/refined/diabetes in pregnancy.txt", "/content/drive/MyDrive/refined/Diabetes medication_TXT.txt" , "/content/drive/MyDrive/refined/diabetes monitoring_TXT.txt" , "/content/drive/MyDrive/refined/Diabetes Nutrition.txt", "/content/drive/MyDrive/refined/diabetes stress mgmt_TXT.txt" , "/content/drive/MyDrive/refined/Diabetes symptoms.txt", "/content/drive/MyDrive/refined/Diabetes tech_TXT.txt", "/content/drive/MyDrive/refined/Effectiveness of diabetes education and awareness of diabetes mellitus  in  combating  diabetes  in  the  United  Kigdom;  a  literature review .txt", "/content/drive/MyDrive/refined/gulocose monitoring.txt", "/content/drive/MyDrive/refined/KnowledgeofDMgeneralpublicKL.txt", "/content/drive/MyDrive/refined/Public knowledge and awareness of diabetes mellitus, its risk factors, complications, and prevention methods among adults in Poland—A 2022 nationwide cross-sectional survey _ Enhanced Reader.txt", "/content/drive/MyDrive/refined/Type 1 diabetes.txt"]  # Add your file paths here
raw_documents = []
for file_path in file_paths:
    loader = TextLoader(file_path)
    raw_documents.extend(loader.load())


In [23]:
#from langchain.document_loaders import WikipediaLoader
#raw_documents = WikipediaLoader(query="Diabetes_management").load()

In [24]:
len(raw_documents)

19

In [25]:
raw_documents[:3]

[Document(metadata={'source': '/content/drive/MyDrive/refined/20121CoffmanNortonBeene.txt'}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/225051151\nDiabetes symptoms, health literacy, and health care use in adult Latinos with\ndiabetes risk factors\nArticle\xa0\xa0in\xa0\xa0Journal of Cultural Diversity · March 2012\nSource: PubMed\nCITATIONS\n26\nREADS\n2,201\n3 authors, including:\nMaren J Coffman\nUniversity of North Carolina at Charlotte\n42 PUBLICATIONS\xa0\xa0\xa0725 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Maren J Coffman on 15 November 2015.\nThe user has requested enhancement of the downloaded file.\nMAREN J. COFFMAN, P H D , R N ,\nCYNTHIA K. NORTON, MSN, ANP-BC, LARA BEENE, M F A\nAbstract: Purpose: This study examined the extent of undetected and un-\ntreated type 2 diabetes in a sample of recent Latino immigrants, and looked\nat the relationships among

In [26]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [33]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

In [34]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [29]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [30]:
graph_documents

[GraphDocument(nodes=[Node(id='Diabetes', type='Health condition'), Node(id='Health Literacy', type='Concept'), Node(id='Health Care', type='Concept'), Node(id='Adult Latinos', type='Demographic group'), Node(id='Maren J Coffman', type='Person'), Node(id='Cynthia K Norton', type='Person'), Node(id='Lara Beene', type='Person')], relationships=[Relationship(source=Node(id='Diabetes', type='Health condition'), target=Node(id='Adult Latinos', type='Demographic group'), type='AFFECTS'), Relationship(source=Node(id='Diabetes', type='Health condition'), target=Node(id='Health Literacy', type='Concept'), type='AFFECTS'), Relationship(source=Node(id='Diabetes', type='Health condition'), target=Node(id='Health Care', type='Concept'), type='AFFECTS'), Relationship(source=Node(id='Maren J Coffman', type='Person'), target=Node(id='University Of North Carolina At Charlotte', type='Organization'), type='AFFILIATED_WITH'), Relationship(source=Node(id='Cynthia K Norton', type='Person'), target=Node(id=

In [31]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [18]:
import pandas as pd


In [19]:
graph

In [65]:
#Define the prompt for entity extraction
class Entities(BaseModel):
    """Identifying information about entities."""
    names: List[str] = Field(
        ..., description="Entities that appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting entities related to the given text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Read questions from the Excel file
questions_df = pd.read_excel('/content/drive/MyDrive/refined/RAGTest.xlsx')  # Replace with your file path
questions = questions_df['Question'].tolist()

# Prepare a list to store generated answers
generated_answers = []

# Generate answers for each question
for question in questions:
    entities = entity_chain.invoke({"question": question}).names
    answer = ""
    for entity in entities:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        answer += "\n".join([el['output'] for el in response])
    generated_answers.append(answer)

# Write generated answers back to the Excel file
questions_df['Generated Answer'] = generated_answers
questions_df.to_excel('/content/drive/MyDrive/refined/questions_with_answers.xlsx', index=False)

# Optionally, compare generated answers with human-generated responses
human_answers = questions_df['Human Answer'].tolist()
questions_df['Accuracy'] = questions_df.apply(lambda row: 1 if row['Generated Answer'] == row['Human Answer'] else 0, axis=1)
accuracy = questions_df['Accuracy'].mean()

print(f"Accuracy of generated answers: {accuracy * 100:.2f}%")

Accuracy of generated answers: 0.00%


In [67]:
import pandas as pd
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

# Define the prompt for entity extraction
class Entities(BaseModel):
    """Identifying information about entities."""
    names: List[str] = Field(
        ..., description="Entities that appear in the text",
    )

# Initialize your LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

# Define the prompt template for extracting entities
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are extracting entities related to the given text."),
        ("human", "Use the given format to extract information from the following input: {question}"),
    ]
)

# Create an entity chain using the LLM
entity_chain = prompt | llm.with_structured_output(Entities)

def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    return full_text_query.strip()

# Read questions from the Excel file
questions_df = pd.read_excel('/content/drive/MyDrive/refined/RAGTest.xlsx')  # Replace with your file path
questions = questions_df['Question'].tolist()

# Prepare a list to store generated answers
generated_answers = []

# Generate answers for each question
for question in questions:
    # Extract entities related to the question
    entities = entity_chain.invoke({"question": question}).names
    answer = ""
    for entity in entities:
        # Query the graph for each entity
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        # Join results into a single answer string
        answer += "\n".join([el['output'] for el in response])
    generated_answers.append(answer or "No relevant information found.")

# Write generated answers back to the Excel file
questions_df['Generated Answer'] = generated_answers
questions_df.to_excel('/content/drive/MyDrive/refined/questions_with_answers.xlsx', index=False)

# Optionally, compare generated answers with human-generated responses
if 'Human Answer' in questions_df.columns:
    human_answers = questions_df['Human Answer'].tolist()
    questions_df['Accuracy'] = questions_df.apply(
        lambda row: 1 if row['Generated Answer'].strip() == row['Human Answer'].strip() else 0,
        axis=1
    )
    accuracy = questions_df['Accuracy'].mean()
    print(f"Accuracy of generated answers: {accuracy * 100:.2f}%")
else:
    print("Human Answer column not found in the Excel file.")


Accuracy of generated answers: 0.00%


In [72]:
import pandas as pd
from typing import List, Tuple
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
import os

# Define the prompt for entity extraction
class Entities(BaseModel):
    """Identifying information about entities."""
    names: List[str] = Field(
        ..., description="Entities that appear in the text",
    )

# Initialize LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

# Define the prompt template for extracting entities
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are extracting entities related to the given text."),
        ("human", "Use the given format to extract information from the following input: {question}"),
    ]
)

# Create an entity chain using the LLM
entity_chain = prompt | llm.with_structured_output(Entities)

def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    return full_text_query.strip()

def get_comprehensive_answer(question: str) -> str:
    # Extract entities related to the question
    entities = entity_chain.invoke({"question": question}).names

    if not entities:
        return "No relevant information found."

    answer_parts = []

    for entity in entities:
        # Query the graph for each entity
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )

        # Process the response to construct a complete answer
        results = [el['output'] for el in response]
        if results:
            answer_parts.append(f"Information related to '{entity}':\n" + "\n".join(results))
        else:
            answer_parts.append(f"No information found for '{entity}'.")

    return "\n\n".join(answer_parts)

# Function to retrieve both structured and unstructured data
def retriever(question: str) -> str:
    print(f"Search query: {question}")
    structured_data = get_comprehensive_answer(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"\n".join(unstructured_data)}
   """,
    return final_data

# Define the template for final answer generation
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

prompt = ChatPromptTemplate.from_template(template)

# Chain to handle the input question and context
chain = (
    RunnableParallel(
        {
            "context": retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

# Read questions from the Excel file
questions_df = pd.read_excel('/content/drive/MyDrive/refined/RAGTest.xlsx')  # Replace with your file path
questions = questions_df['Question'].tolist()

# Prepare a list to store generated answers
generated_answers = []

# Generate answers for each question
for question in questions:
    answer = chain.invoke({"question": question})
    generated_answers.append(answer)

# Write generated answers back to the Excel file
questions_df['Generated Answer'] = generated_answers
questions_df.to_excel('/content/drive/MyDrive/refined/questions_with_answers.xlsx', index=False)

# Optionally, compare generated answers with human-generated responses
if 'Human Answer' in questions_df.columns:
    human_answers = questions_df['Human Answer'].tolist()
    questions_df['Accuracy'] = questions_df.apply(
        lambda row: 1 if row['Generated Answer'].strip() == row['Human Answer'].strip() else 0,
        axis=1
    )
    accuracy = questions_df['Accuracy'].mean()
    print(f"Accuracy of generated answers: {accuracy * 100:.2f}%")
else:
    print("Human Answer column not found in the Excel file.")


SyntaxError: f-string expression part cannot include a backslash (<ipython-input-72-6f5aa3105607>, line 86)

In [20]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"
default_cypher

'MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50'

In [21]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [22]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [23]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [24]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [25]:
from typing import Tuple, List, Optional

In [26]:
from langchain_community.vectorstores import Neo4jVector

In [27]:
from langchain_openai import OpenAIEmbeddings
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [28]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [29]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the diabetes care related entities that "
        "appear in the text",
    )


In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [31]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting symptoms and care related entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [35]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [36]:
entity_chain.invoke({"question": "At what age diabetes risk gets increased?"}).names

['diabetes risk', 'age']

In [37]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [38]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [39]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [40]:
print(structured_retriever("What Diabetes is?"))

Diabetic Adults - PUBLISHED_IN -> Dodd, M.
Diabetic Adults - PUBLISHED_IN -> Doty, M. M.
Diabetic Adults - PUBLISHED_IN -> Garcia, A. A.
Diabetic Adults - PUBLICATION -> Dodd, M.
Diabetic Adults - PUBLICATION -> Doty, M. M.
Diabetic Adults - PUBLICATION -> Garcia, A. A.
Diabetes - AFFECTS -> Health Literacy
Diabetes - AFFECTS -> Health Care
Diabetes - AFFECTS -> Adult Latinos
Diabetes - AFFECTS -> Participants
Diabetes - RESEARCHER -> Maren J Coffman
Diabetes - RESEARCHER -> Cynthia K Norton
Diabetes - RESEARCHER -> Lara Beene
Diabetes - RELATED_TO -> Blood Glucose Level
Diabetes - RELATED_TO -> Diabetes Knowledge
Diabetes - RELATED_TO -> Health Literacy Levels
Diabetes - RELATED_TO -> Health Care Services
Diabetes - RELATED_TO -> Undiagnosed Diabetes
Diabetes - RELATED_TO -> Under Treated Diabetes
Diabetes - RELATED_TO -> Diabetes Symptoms
Diabetes - ASSOCIATED_WITH -> Education Level
Diabetes - WITH -> Diabetes Risk Factors
Diabetes - NOT_COMPARABLE_TO -> Diagnosed Diabetes
Diabetes 

In [41]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [42]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [43]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [44]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [45]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [46]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [47]:
prompt = ChatPromptTemplate.from_template(template)

In [48]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [49]:
chain.invoke({"question": "how stress-related hormones affects diabetes and sugar level?"})

Search query: how stress-related hormones affects diabetes and sugar level?


'Stress-related hormones can affect diabetes by increasing blood sugar levels. This is because stress triggers the release of hormones like cortisol and adrenaline, which can cause blood sugar levels to rise. High levels of stress can make it harder to manage diabetes and control blood sugar levels.'

In [50]:
chain.invoke(
    {
        "question": "When diabetes gets worst",
        "chat_history": [("how stress-related hormones affects diabetes and sugar level?", "increases diabetes")],
    }
)

Search query: When does diabetes get worse?


'When diabetes worsens, it can lead to more severe symptoms and complications.'

In [51]:
#import pandas as pd
#from pydantic import BaseModel, Field
#from typing import List
#from langchain_openai import ChatOpenAI
#from langchain.prompts import ChatPromptTemplate
#from langchain_experimental.graph_transformers import LLMGraphTransformer
#from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars


# Assuming RunnableParallel and StrOutputParser exist and are properly imported
chain = (
    RunnableParallel(
        {
            "context":  _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

# Example to test with a single question
result = chain.invoke({"question": "how stress-related hormones affects diabetes and sugar level?"})
print(result)



Search query: how stress-related hormones affects diabetes and sugar level?
Stress-related hormones can affect diabetes by increasing blood sugar levels. This is because stress triggers the release of hormones like cortisol and adrenaline, which can cause blood sugar levels to rise. This can be problematic for individuals with diabetes, as it can lead to difficulty in managing their blood sugar levels effectively.


In [52]:
# Read questions from the Excel file
questions_df = pd.read_excel('/content/drive/MyDrive/refined/RAGTest.xlsx')  # Replace with your file path
questions = questions_df['Question'].tolist()

# Prepare a list to store generated answers
generated_answers = []

# Iterate through questions and generate answers
for question in questions:
    answer = chain.invoke({"question": question})
    generated_answers.append(answer)

# Write generated answers back to the Excel file
questions_df['Generated Answer'] = generated_answers
questions_df.to_excel('/content/drive/MyDrive/refined/questions_with_answers.xlsx', index=False)

# Optionally, compare generated answers with human-generated responses
if 'Human Answer' in questions_df.columns:
    human_answers = questions_df['Human Answer'].tolist()
    questions_df['Accuracy'] = questions_df.apply(
        lambda row: 1 if row['Generated Answer'].strip() == row['Human Answer'].strip() else 0,
        axis=1
    )
    accuracy = questions_df['Accuracy'].mean()
    print(f"Accuracy of generated answers: {accuracy * 100:.2f}%")
else:
    print("Human Answer column not found in the Excel file.")

Search query: What are some early warning signs of diabetes that often go unnoticed?
Search query: How can intermittent fasting affect blood sugar levels in diabetic patients?
Search query: What role does sleep quality play in the management of Type 2 diabetes?
Search query: How can diabetic patients incorporate mindful eating practices to better manage their condition?
Search query: What are the benefits of using a hybrid closed-loop insulin pump system?
Search query: How does stress impact blood glucose levels, and what are effective stress management techniques for diabetics?
Search query: Can consuming probiotics and prebiotics improve insulin sensitivity in diabetic patients?
Search query: What are some innovative approaches to diabetes education that have shown promise in recent studies?
Search query: How can a diabetic patient safely participate in endurance sports or high-intensity interval training (HIIT)?
Search query: What are the latest advancements in artificial pancreas s

In [54]:
questions_df['Generated Answer'][0]

'Some early warning signs of diabetes that often go unnoticed include misinterpreting serious symptoms as benign, not recognizing symptoms as related to diabetes, and self-treating symptoms without seeking professional assessment.'